In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) <100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,1191655.83,-13.64,4.19,-10.12,-0.21,-0.28,-0.01
4,0.0,-13.64,1271179.36,-14.90,-9.39,6.45,0.55,0.14
5,0.0,4.19,-14.90,363011.78,1.40,2.64,1.46,-0.12
6,0.0,-10.12,-9.39,1.40,119355.95,-0.06,-0.04,-0.03
7,0.0,-0.21,6.45,2.64,-0.06,25375.06,0.13,0.04
8,0.0,-0.28,0.55,1.46,-0.04,0.13,2232.18,0.01
9,0.0,-0.01,0.14,-0.12,-0.03,0.04,0.01,107.69


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_20596/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8,9
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,-0.00001,0.00001,-0.00003,-0.00000,-0.00001,-0.00000
4,NaN,-0.00001,1.00000,-0.00002,-0.00002,0.00004,0.00001,0.00001
5,NaN,0.00001,-0.00002,1.00000,0.00001,0.00003,0.00005,-0.00002
6,NaN,-0.00003,-0.00002,0.00001,1.00000,-0.00000,-0.00000,-0.00001
7,NaN,-0.00000,0.00004,0.00003,-0.00000,1.00000,0.00002,0.00002
8,NaN,-0.00001,0.00001,0.00005,-0.00000,0.00002,1.00000,0.00002
9,NaN,-0.00000,0.00001,-0.00002,-0.00001,0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

36.59325105161713

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.19483000e+11 1.60965889e+11 1.56867629e+10 1.62405063e+09
 6.88216822e+07 4.74385152e+05 8.27843877e+02 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.0,-0.539719,0.841724,-0.013629,-0.004132,-0.000866,-0.000076,-0.000004
1,-0.0,0.839802,0.537152,-0.075410,-0.022186,-0.004655,-0.000410,-0.000020
2,-0.0,0.054723,0.050804,0.994273,-0.075153,-0.013979,-0.001219,-0.000059
3,0.0,0.020309,0.019020,0.072341,0.995363,-0.056777,-0.004714,-0.000228
4,-0.0,0.005353,0.005013,0.017621,0.055291,0.997841,-0.029846,-0.001424
5,-0.0,0.000625,0.000585,0.002045,0.006235,0.029466,0.999251,-0.024207
6,-0.0,0.000045,0.000042,0.000149,0.000452,0.002121,0.024153,0.999706
7,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.16019823626937635,
 0.9491962869192686,
 0.9276585818210445,
 0.9447086136692715,
 0.9705339543956434,
 0.9758473692220205,
 1.0]